<a href="https://colab.research.google.com/github/weiyw16/weiyw16.github.io/blob/main/quick_view_seismic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### ########################## ###
### quick view of seismic data ###
###       support .npy         ###
### Author: Yanwen Wei         ###
### Email: weiyw17@gmail.com   ###
### ########################## ###

import os 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pcl
# from matplotlib.font_manager import FontProperties # if use cn
# add your font 
# SimHei = FontProperties(fname='/home/wyw/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf')

# global paramaters
# no need to change here
import argparse
opt = argparse.ArgumentParser()
opt.lang = 'en'# 'cn' 
opt.ftype = 0 # 0 for .npy; 1 for .bin
opt.n1 = 2048 # fast dimension 
opt.n2 = 126 # second dimension 2D problem

opt.ss = 50 # start from which data
opt.npic = 10 # how many shots to show per figure

In [ ]:
# change here
### # download data from ###############
# https://pan.baidu.com/s/1xR0sFMzBbdXmg7mf4VPMwQ?pwd=hpgc
# or https://drive.google.com/drive/folders/1J1UI__lw1BjUFeBrUA4ms6QqOXoB5HGS?usp=sharing)
# and put the path here
opt.rpath = '/home/wyw/2021_workspace/B-rename_for_input' 
opt.mode = 'train' # 'test'
# opt.mode = 'test' # 'test'
### # ##################################

# change here: which model to plot
if opt.mode == 'train':
    # train list
#     mid_list = [10, 23, 30, 414, 419, 430, 431, 432, 439, 440, 441, 450, 451, 452, 453, 458, 459, 62, 66, 80]
    mid_list = [52]
else:
    # test list
    # mid_list = [10, 23, 432, 435, 456, 52, 62, 66, 80, 81, 94, 95, 96] 
    mid_list = [10]

## Functions

In [3]:
def get_plot_stack2image(opt=None, OUT=None, scale=0.1, title=None, ratio=16, nplot=None):

    sc1 = OUT.mean() - scale*OUT.std()
    sc2 = OUT.mean() + scale*OUT.std()
    cN = pcl.Normalize(vmin=sc1, vmax=sc2)
    
    fig, ax = plt.subplots(figsize=(8,5), dpi=300)
    im = ax.imshow(OUT, cmap='seismic', norm=cN, interpolation='bicubic')
    ax.set_aspect(ratio)
    if opt.lang == 'cn':
            plt.ylabel(u'时间步', fontsize=10, fontproperties=SimHei) # y轴标题
            plt.xlabel(u'接收器', fontsize=10, fontproperties=SimHei) # y轴标题
    else:
        plt.ylabel(u'Time Step', fontsize=10) # y轴标题
        # plt.gca().invert_yaxis()
        plt.xlabel(u'Receiver No', fontsize=10) # x轴标题
    
    plt.title(title)
    nx = opt.n2
    for ii in range(1, nplot):
        plt.axvline( int((ii)*nx), color='black') #, linewidth=2 )
    return ax 

In [4]:
def routinue_npic(opt=None, path=None, prefix=None, ss=0, npic=10):
    # single component plot 
#     npic = 10
    all_show = np.zeros( (opt.n1, npic*opt.n2) )
    print(f"Reading from {path}")

    for ii in range(npic):
        iis = ii + ss
        tmp1 = np.load(f"{path}/{prefix}_{iis}.npy")
        all_show[:,ii*opt.n2:ii*opt.n2+opt.n2] = tmp1

    ax1 = get_plot_stack2image(opt=opt, OUT=all_show, scale=3, ratio=0.2, nplot=npic)
#     print(f"Start at {ss}")
#     ss += npic;
    return 

In [5]:
def workflow(opt=opt, mid=None):
    
    # set path
    if opt.mode == 'train':
        opt.path_pre = f'{opt.rpath}/for{opt.mode}_model_{mid}'
    else:
        opt.path_pre = f'{opt.rpath}/{opt.mode}_model_{mid}'
    opt.prefix = 'vz'
    opt.path = f'{opt.path_pre}/{opt.prefix}'

    opt.prefix2 = 'vx'
    opt.path2 = f'{opt.path_pre}/{opt.prefix2}'

    opt.prefix3 = 'div'
    opt.path3 = f'{opt.path_pre}/{opt.prefix3}'

    opt.prefix4 = 'curl'
    opt.path4 = f'{opt.path_pre}/{opt.prefix4}'

    opt.outpath = opt.path_pre.split('/')[-1]
    print(f"write out to {opt.outpath}")
    if not os.path.exists(opt.outpath):
        os.mkdir(opt.outpath)
        
    # plot
    ss = opt.ss; npic = opt.npic;
    path = opt.path
    prefix = opt.prefix
    routinue_npic(opt=opt, path=path, prefix=prefix, ss=ss, npic=10)
    plt.title(f"vModel-{mid}-{opt.mode}-shot-{opt.ss}-{opt.ss+npic-1}-{prefix}")
    plt.savefig(f'{opt.outpath}/single_comp_{prefix}_{ss}-{ss+npic}.pdf', bbox_inches='tight')
    
    path = opt.path2
    prefix = opt.prefix2
    routinue_npic(opt=opt, path=path, prefix=prefix, ss=ss, npic=10)
    plt.title(f"vModel-{mid}-{opt.mode}-shot-{opt.ss}-{opt.ss+npic-1}-{prefix}")
    plt.savefig(f'{opt.outpath}/single_comp_{prefix}_{ss}-{ss+npic}.pdf', bbox_inches='tight')

    path = opt.path3
    prefix = opt.prefix3
    routinue_npic(opt=opt, path=path, prefix=prefix, ss=ss, npic=10)
    plt.title(f"vModel-{mid}-{opt.mode}-shot-{opt.ss}-{opt.ss+npic-1}-{prefix}")
    plt.savefig(f'{opt.outpath}/single_comp_{prefix}_{ss}-{ss+npic}.pdf', bbox_inches='tight')
    
    path = opt.path4
    prefix = opt.prefix4
    routinue_npic(opt=opt, path=path, prefix=prefix, ss=ss, npic=10)
    plt.title(f"vModel-{mid}-{opt.mode}-shot-{opt.ss}-{opt.ss+npic-1}-{prefix}")
    plt.savefig(f'{opt.outpath}/single_comp_{prefix}_{ss}-{ss+npic}.pdf', bbox_inches='tight')
    
    ss += npic;
    return 

# data visualization

In [9]:
for mid in mid_list:
    workflow(opt=opt, mid=mid)

write out to fortrain_model_52
Reading from /home/wyw/2021_workspace/B-rename_for_input/fortrain_model_52/vz


FileNotFoundError: ignored